## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "study"
mode_of_interest = None
include_test_users = False
sensed_algo_prefix = "cleaned"

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_sensor_inference_data(year,
                                                                            month,
                                                                            program,
                                                                            include_test_users,
                                                                            sensed_algo_prefix)

## Generic Metrics

### Distribution of Mode_confirm attribute

In [ ]:
file_name='ntrips_sensed_mode%s' % file_suffix
plot_title_no_quality= "Number of trips for each primary mode\n(inferred by OpenPATH from phone sensors)"
try:
    labels_mc = expanded_ct['primary_mode'].value_counts(dropna=True).keys().tolist()
    values_mc = expanded_ct['primary_mode'].value_counts(dropna=True).tolist()    
    plot_title = plot_title_no_quality+"\n"+quality_text
    pie_chart_sensed_mode(plot_title,labels_mc,values_mc,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_mc, labels_mc), file_name, plot_title)
    print(expanded_ct['primary_mode'].value_counts(dropna=True))
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Mode choice for trips under 10 miles

In [ ]:
plot_title_no_quality="Number of trips under 10 Miles for each primary mode\n(inferred by OpenPATH from phone sensors)"
file_name ='ntrips_under10miles_sensed_mode%s' % file_suffix
try:
    labels_d10 = expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].primary_mode.value_counts(dropna=True).keys().tolist()
    values_d10 = expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].primary_mode.value_counts(dropna=True).tolist()
    d10_quality_text = scaffolding.get_quality_text(expanded_ct, expanded_ct[expanded_ct['distance_miles'] <= 10], "< 10 mile", include_test_users)
    plot_title= plot_title_no_quality+"\n"+d10_quality_text
    pie_chart_sensed_mode(plot_title,labels_d10,values_d10,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_d10, labels_d10), file_name, plot_title)
    print(expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].primary_mode.value_counts(dropna=True))
except:
    d10_df = expanded_ct.query("distance_miles <= 10") if "distance_miles" in expanded_ct.columns else expanded_ct
    debug_df.loc["Trips_less_than_10_miles"] = scaffolding.trip_label_count("Mode_confirm", d10_df)
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Miles per chosen transport mode

In [ ]:
plot_title_no_quality="Miles for each primary mode\n(inferred by OpenPATH from phone sensors)"
file_name ='miles_sensed_mode%s' % file_suffix

try:
    miles = expanded_ct.groupby('primary_mode').agg({'distance_miles': ['sum', 'count' , 'mean']})
    miles.columns = ['Total (miles)', 'Count', 'Average (miles)']
    miles = miles.reset_index()
    miles =miles.sort_values(by=['Total (miles)'], ascending=False)

    #data
    miles_dict = dict(zip(miles['primary_mode'], miles['Total (miles)']))

    labels_m = []
    values_m = []

    for x, y in miles_dict.items():
        labels_m.append(x)
        values_m.append(y)
        
    plot_title="Miles for each mode\n(inferred by OpenPATH from phone sensors)\n%s" % quality_text

    pie_chart_sensed_mode(plot_title,labels_m,values_m,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_m, labels_m), file_name, plot_title)
    print(miles)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Miles per chosen land transport mode

In [ ]:
plot_title_no_quality="Miles for each land-based primary mode\n(inferred by OpenPATH from phone sensors)"
file_name ='miles_sensed_mode_land%s' % file_suffix

try:
    miles = expanded_ct.groupby('primary_mode').agg({'distance_miles': ['sum', 'count' , 'mean']})
    miles.columns = ['Total (miles)', 'Count', 'Average (miles)']
    miles = miles.reset_index()
    miles =miles.sort_values(by=['Total (miles)'], ascending=False)

    #data
    miles_dict = dict(zip(miles['primary_mode'], miles['Total (miles)']))

    labels_m = []
    values_m = []

    for x, y in miles_dict.items():
        if x != "AIR_OR_HSR":
            labels_m.append(x)
            values_m.append(y)
        
    plot_title="Miles for each land-based mode\n(inferred by OpenPATH from phone sensors)\n%s" % quality_text

    pie_chart_sensed_mode(plot_title,labels_m,values_m,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_m, labels_m), file_name, plot_title)
    print(miles)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Average miles per transport mode selected (primary_mode)

In [ ]:
plot_title_no_quality=" Average Miles for each mode with > 3 entries"
file_name ='average_miles_sensed_mode%s' % file_suffix

try:
    data = miles.drop((miles.query("Count < 3").index)).sort_values(by=['Average (miles)'], ascending=False)
    x='primary_mode'
    y='Average (miles)'
    
    plot_title= plot_title_no_quality+"\n"+quality_text
    
    print(expanded_ct['primary_mode'].dropna().unique().tolist())
    barplot_sensed_mode(data,x,y,plot_title, expanded_ct['primary_mode'].dropna().unique().tolist(), file_name)
    print("Next")
    alt_text = store_alt_text_bar(pd.DataFrame(data['Average (miles)'].values, data['primary_mode']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality) 

### Number of trips by day¶

In [ ]:
plot_title_no_quality="Number of trips by day\n(inferred by OpenPATH from phone sensors)"
file_name ='ntrips_sensed_per_day%s' % file_suffix

try:
    fq_days = expanded_ct.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
    fq_days = fq_days.reset_index()
    fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

    data = fq_days
    x = 'Day of the Month'
    y = 'Number of Trips'
    
    plot_title= plot_title_no_quality+"\n"+quality_text

    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Number of trips by day of week¶

In [ ]:
plot_title_no_quality="Number of trips by weekday\n(inferred by OpenPATH from phone sensors)"
file_name ='ntrips_sensed_per_weekday%s' % file_suffix
try:
    fq_weekdays = expanded_ct.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
    fq_weekdays = fq_weekdays.reset_index()
    fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
    weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

    data = fq_weekdays
    x = 'Weekday'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    